# 2021 U.S. Traffic Accident Analysis

## Overview

Our project is to uncover patterns in traffic accidents across the US.
We'll examine relationships between location and volume and severity of accidents; impact of disruptions to traffic 
flow to accident severity; effects of weather on traffic safety; and related questions, as the data admits. 

In [ ]:
#Import dependencies
import csv
import pandas as pd
import matplotlib.pyplot as plt
import gmaps
import gmaps.datasets
import requests
import json
import seaborn as sns
from config import gkey 
from random import sample
import warnings
warnings.filterwarnings('ignore')
import scipy.stats as stats
from scipy.stats import linregress
import numpy as np

In [ ]:
#Load CSV
accident_data = pd.read_csv("Resources/2021_us_accidents.csv")
accident_data.head()

In [ ]:
#Add a column to count each accident as 1
accident_data["# of accidents"] = 1
accident_data.head()

## Accidents by Location

#### How many accidents happened in 2021? 
* There were about 1.5 Million accidents reported in the US in 2021

#### Which states have the highest and lowest accident total?
* The three states with the highest recorded number of reported accidents were California, Florida, and Texas.
* The three states with the lowest recorded number of reported accidents were Maine, Vermont, and South Dakota.

In [ ]:
#See how many rows there are
len(accident_data["State"])

In [ ]:
#Pull out columns for the heatmap
accident_data[["Start_Lat","Start_Lng","State","Severity"]][724:737]

In [ ]:
#Create a sample for the limitations of our API key
sample = accident_data.sample(n = 10 , replace = False)
print(sample)

In [ ]:
ustates = []
for x in accident_data["State"]:
    if x not in ustates:
        ustates.append(x)
print(ustates)
#This list includes the continential US and Washington DC. (Excludes AK and HI)

In [ ]:
fig, ax = plt.subplots(figsize = (15.7,10.27))
_severity = ['1', '2', '3', '4']


sns.scatterplot(data = accident_data, x = "Start_Lng", y = "Start_Lat", hue = 'Severity', palette = ['green','blue'])

plt.title("Accidents in US by Severity", fontsize = 20)
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.grid()

In [ ]:
#Sum of accident for each state
ustates = []
for x in accident_data["State"]:
    if x not in ustates:
        ustates.append(x)
for x in ustates:
    state_data = accident_data.loc[accident_data["State"] == x]
    print(f'{x} : {state_data["# of accidents"].sum()}')

In [ ]:
#Create a data frame of the top 3 states with the most accidents
top_three = accident_data[["Start_Lat","Start_Lng","State","# of accidents"]]
top_three.reset_index()

reduced_top = top_three.loc[(top_three["State"] == "CA") | (top_three["State"] == "FL") | (top_three["State"] == "TX")]
reduced_top

In [ ]:
#Create an equal sample of each state to fit the limitations of the google heat map
ca_sample = reduced_top.loc[reduced_top["State"] == "CA"]
reduced_ca_sample = ca_sample.sample(n=10)

fl_sample = reduced_top.loc[reduced_top["State"] == "FL"]
reduced_fl_sample = fl_sample.sample(n=10)

tx_sample = reduced_top.loc[reduced_top["State"] == "TX"]
reduced_tx_sample = tx_sample.sample(n=10)

In [ ]:
#Combine CA and FL
cafl_sample = pd.merge(reduced_ca_sample,reduced_fl_sample, on=["State","Start_Lat","Start_Lng","# of accidents"], how= "left")
cafl_sample

In [ ]:
#Combine the CAFL with TX to get a data frame of all three states
cafl_sample = pd.merge(reduced_ca_sample,reduced_fl_sample, on=["State","Start_Lat","Start_Lng","# of accidents"], how= "outer")
three_states = pd.merge(cafl_sample,reduced_tx_sample, on= ["State","Start_Lat","Start_Lng","# of accidents"], how= "outer")
three_states.head()

In [ ]:
# #Code for heatmap to show top 3 states with the most accidents
# gmaps_key = gmaps.configure(api_key = gkey)
# locations_top3 = three_states[["Start_Lat","Start_Lng"]]
# weights_top3 = three_states["# of accidents"]

In [ ]:
# #Code to display heatmap
# fig5 = gmaps.figure()
# fig5.add_layer(gmaps.heatmap_layer(locations_top3, weights = weights_top3))
# fig5

In [ ]:
# #Code for heatmap of sample across the nation
# gmaps_key = gmaps.configure(api_key = gkey)
# locations = sample[["Start_Lat","Start_Lng"]]
# weights = sample["# of accidents"][0:10]

In [ ]:
# #Code to display heatmap
# fig6 = gmaps.figure()
# fig6.add_layer(gmaps.heatmap_layer(locations, weights = weights))
# fig6

In [ ]:
# Create a bar chart to show the accidents and severity by state
accbystate = accident_data[["State", "Severity","# of accidents"]]
accbystate.groupby(['State', 'Severity']).size().unstack().plot(kind = 'barh', stacked = True, figsize = (15, 11))
plt.title("Accidents and Severity by State", fontsize = 20)
plt.xlabel("Number of Accidents")
plt.tight_layout

In [ ]:
sns.set_theme(style="darkgrid")
fig, ax = plt.subplots(figsize=(11.7,6.27))
sns.countplot(y = "State", data = accident_data, order=accident_data.State.value_counts().iloc[:10].index).set_title('Top 10 States with the most Accidents', fontsize = 20);

## Visibility (Weather and Daylight)

#### How does visibility affect likelihood of accidents?
##### Daylight vs. Nighttime
 * 64.62% of accidents happened during the day and only 35.38% happened at night. Although night driving is thought to be more dangerous, there is much more traffic during the day.
 
##### Weather visibility
 * The number of accidents vs visibility yielded a positive relationship. We have $r^2 \approx 0.5940$ which corresponds to a moderate fit.

In [ ]:
vis_data = accident_data.groupby("Visibility(mi)").count()
vis_data["# of accidents"]

In [ ]:
#Create a data frame to be used in the scatter plot
vis_data.reset_index(inplace = True)
vis_data.head()

In [ ]:
#Scatter plot of visibility vs accidents
groupby = accident_data.groupby(['Visibility(mi)'])['# of accidents'].sum()
x_values = vis_data["Visibility(mi)"]
y_values = vis_data["# of accidents"]
plt.scatter(x_values,y_values)
plt.xlabel("Visibility(mi)")
plt.ylabel("Number of accidents")
plt.title("Visibility vs Accidents", fontsize = 20)
fig1 = plt.gcf()
fig1.set_size_inches(15.7,10.27)
plt.show()


In [ ]:
adj_vis_data = accident_data[["Visibility(mi)","# of accidents"]]
#Create adjusted visilibity data so anything over 10 will now read 10
adj_vis_data["Adjusted_Visibility"] = adj_vis_data["Visibility(mi)"]
adj_vis_data.loc[adj_vis_data["Visibility(mi)"] > 10, "Adjusted_Visibility"] = 10
#Create adjusted visilibity data so anything 0-.25 = .25
adj_vis_data.loc[adj_vis_data["Visibility(mi)"] <= .25, "Adjusted_Visibility"] = .25
#Create adjusted visilibity data so anything .26-.50 = .5
adj_vis_data.loc[adj_vis_data["Visibility(mi)"].between(.25,.5), "Adjusted_Visibility"] = .5
#Create adjusted visilibity data so anything .51-.75=.75
adj_vis_data.loc[adj_vis_data["Visibility(mi)"].between(.5,.75), "Adjusted_Visibility"] = .75
#Create adjusted visilibity data so anything .76-1=1
adj_vis_data.loc[adj_vis_data["Visibility(mi)"].between(.75,1), "Adjusted_Visibility"] = 1
#Create new dataframe with only Adjusted Visibility under 10
adj_vis_data = adj_vis_data.loc[adj_vis_data["Visibility(mi)"] < 10]
adj_vis_data.head(10)

In [ ]:
#Group and count accidents per visibility
adj_vis_data_group = adj_vis_data.groupby("Adjusted_Visibility").count()
adj_vis_data_group.reset_index(inplace = True)
adj_vis_data_group

In [ ]:
#Plot the scatter plot for adjusted visibility and # of accidents
x_values = adj_vis_data_group["Adjusted_Visibility"]
y_values = adj_vis_data_group["# of accidents"]
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(6,12000),fontsize=15,color="red")
plt.xlabel("Adjusted Visibility(mi)")
plt.ylabel("# of accidents")
plt.title("Adjusted Visibility and Number of Accidents", fontsize = 20)
print(f"The r-squared is: {rvalue**2}")
fig3 = plt.gcf()
fig3.set_size_inches(15.7,10.27)
plt.show()

In [ ]:
#Create and format a pie chart for day vs night accidents
daynight = accident_data[["Sunrise_Sunset","# of accidents"]].groupby("Sunrise_Sunset").count()
colors2 = ["lightblue","orange"]
daynight.plot.pie(y = "# of accidents",colors = colors2 , legend = False, autopct = "%1.2f%%")
fig4 = plt.gcf()
fig4.set_size_inches(15.7,10.27)
plt.title("Number of Accidents Day vs Night", fontsize = 20)

## Traffic Distruptions

**Alternative Hypothesis:** Traffic obstructions help to reduce the number of accidents.

**Null Hypothesis:** Traffic obstructions have no effect on the number of accidents.

**Chi-Squared Test**
#### How do traffic distruptions affect the total number of accidents?
* 83.4% of all traffic accidents in 2021 were not near a traffic obstruction (signal, stop sign, roundabout, or crosswalk)
* Only 0.006% of accidents happened at a roundabout, 2% at a stopsign, 7.2% at a crosswalk, and 7.4% at a traffic signal.
* Using a Chi-squared hypothesis test, we were able to determine that fewer accidents occur near traffic obstruction, p-value near 0.

In [ ]:
#Create a data frame to use for distrubtions pie chart
day_disruptions = accident_data[["Sunrise_Sunset", "Stop", "Roundabout", "Traffic_Signal", "Crossing", "# of accidents"]]
day_disruptions.loc[day_disruptions["Sunrise_Sunset"] =='Day', "Sunrise_Sunset"] = 1
day_disruptions.loc[day_disruptions["Sunrise_Sunset"] =='Night', "Sunrise_Sunset"] = 0
day_disruptions.head(5)

In [ ]:
roundabouts = (day_disruptions.loc[day_disruptions["Roundabout"] == True]).sum()
stop = (day_disruptions.loc[day_disruptions["Stop"] == True]).sum()
signal = (day_disruptions.loc[day_disruptions["Traffic_Signal"] == True]).sum()
crosswalk = (day_disruptions.loc[day_disruptions["Crossing"] == True]).sum()
none = day_disruptions.loc[(day_disruptions["Roundabout"] == False) & (day_disruptions["Stop"] == False) & (day_disruptions["Traffic_Signal"] == False) & (day_disruptions["Crossing"] == False)].sum()

In [ ]:
distruptions = [stop["Stop"], signal["Traffic_Signal"],  roundabouts["Roundabout"], crosswalk["Crossing"], none["# of accidents"]]
distruption_labels = ["Stop Sign","Traffic Signal", "Roundabout", "Crosswalk", "No Disruption"]
colors = ["pink","yellow", "lightgreen", "lightblue", "orange"]
explode = (0.3,0.3,0.3,0.3,0.1)
plt.pie(distruptions, explode=explode, labels=distruption_labels, colors=colors, autopct="%1.2f%%", shadow=True)
fig = plt.gcf()
fig.set_size_inches(10,10)
plt.title("Accidents by Disruption", fontsize = 20)

In [ ]:
labels = ['Crosswalk','Roundabout', 'Traffic Signal', 'Stop Sign']
# Get data for each disruption and Day/Night
x = np.arange(4)
y1 = crosswalk["Sunrise_Sunset"],roundabouts["Sunrise_Sunset"], signal["Sunrise_Sunset"], stop["Sunrise_Sunset"]
y2 =  crosswalk["Crossing"]-crosswalk["Sunrise_Sunset"],roundabouts["Roundabout"]-roundabouts["Sunrise_Sunset"], signal["Traffic_Signal"]-signal["Sunrise_Sunset"], stop["Stop"]-stop["Sunrise_Sunset"]
width = 0.40
# plot data in grouped manner of bar type
plt.bar(x-0.2, y1, width)
plt.bar(x+0.2, y2, width)
plt.ylabel('# of Accidents')
plt.title('Number of Accidents by Disruption and Day/Night', fontsize = 20)
plt.xticks(x, labels)
plt.legend(["Day", "Night"])
fig4 = plt.gcf()
fig4.set_size_inches(15.7,10.27)
plt.show()

In [ ]:
# Declare Variables
accidents_roundabout = roundabouts["# of accidents"]
accidents_stop = stop["# of accidents"]
accidents_signal = signal["# of accidents"]
accidents_crosswalk = crosswalk["# of accidents"]
accidents_no_obstruction = none["# of accidents"]
print(accidents_roundabout)
print(accidents_stop)
print(accidents_signal)
print(accidents_crosswalk)
print(accidents_no_obstruction)
# Calculate total number of accidents and the expected number of accidents
accidents_total = accidents_roundabout + accidents_stop + accidents_signal + accidents_crosswalk + accidents_no_obstruction
expected = accidents_total / 5
print(accidents_total)
print(expected)

In [ ]:
# Create a dataframe with number of observed and expected accidents
accidents_obstruction_df = pd.DataFrame({
    "Observed": [accidents_roundabout, accidents_stop, accidents_signal, accidents_crosswalk,accidents_no_obstruction],
    "Expected": [expected, expected, expected, expected, expected]
})
accidents_obstruction_df

In [ ]:
# Run the chi square test with stats.chisquare()
stats.chisquare(accidents_obstruction_df['Observed'], accidents_obstruction_df['Expected'])

## Analysis

#### How many accidents happened in 2021? 
* There were about 1.5 Million accidents reported in the US in 2021

#### Which states have the highest and lowest accident total?
* The three states with the highest recorded number of reported accidents were California, Florida, and Texas.
* The three states with the lowest recorded number of reported accidents were Maine, Vermont, and South Dakota.

#### How does visibility affect likelihood of accidents?
##### Daylight vs. Nighttime
 * 64.62% of accidents happened during the day and only 35.38% happened at night. Although night driving is thought to be more dangerous, there is much more traffic during the day.
 
##### Weather visibility
 * The number of accidents vs visibility yielded a positive relationship. We have $r^2 \approx 0.5940$ which corresponds to a moderate fit.

#### How do traffic distruptions affect the total number of accidents?
* 83.4% of all traffic accidents in 2021 were not near a traffic obstruction (signal, stop sign, roundabout, or crosswalk)
* Only 0.006% of accidents happened at a roundabout, 2% at a stop sign, 7.2% at a crosswalk, and 7.4% at a traffic signal.
* Using a Chi-squared hypothesis test, we were able to determine that fewer accidents occur near traffic obstruction, p-value near 0.

## Limitations
* Alaska and Hawaii didn't report any traffic accident data.
* Size restriction for GitHub uploads. A lot of potentially interesting data needed to be cut out of the study.
* Traffic volume wasn't included in the data set.
* Visibility varied widely in terms of how it was reported.  In order to standardize, we eliminated 10 mi and above from the analysis as no impact.  Adjusted visibility values were calculated by rounding values under 1 mi up to the nearest quarter mile.
* 200,000 invalid Weather data.
* Used a subset of traffic distruptions in order to reduce our data size.
* Severity was related to the level of impact on traffic
* Very little diversity of severity in our dataset

## Lessons Learned
 - Checked to see if the dataset is not zipped. It can mislead you on the actual size if it is.
 - Having some point of reference for total miles of roadway or traffic volume
 - Looking back we could've done more with our time of day data or day in year data for a better correlation on when accidents will happen.